In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
LSTM_model = tf.keras.models.load_model('acc_gyro_LSTM9.h5')
checkpoint_path = "trainingAG_LSTM_FINAL9/cp.ckpt"
LSTM_model.load_weights(checkpoint_path)

run_model = tf.function(lambda x: LSTM_model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 30
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], LSTM_model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm9"
LSTM_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

In [2]:
dataset = pd.read_csv('dataAccelGyroX3.txt')
dataset.head()

,-5700,6888,12384,420,-5707,-1692,-5980,6384,12776,794,...,1594,-8823,-4679,-2184,14992,1132,1294,-4062,-1328,0
0,-5980,6384,12776,794,-1036,-1222,-7348,7424,13836,730,...,1294,-4062,-1328,-1380,18304,1952,-906,-2616,1565,0
1,-7348,7424,13836,730,-2235,389,-5584,6744,13868,109,...,-906,-2616,1565,-1512,16124,2680,-753,2345,2680,0
2,-5584,6744,13868,109,-2177,-1112,-6224,6480,14060,-137,...,-753,2345,2680,-1576,14240,1920,-1713,550,3010,0
3,-6224,6480,14060,-137,1387,593,-6492,6288,13244,941,...,-1713,550,3010,-932,14812,2484,-1549,989,1968,0
4,-6492,6288,13244,941,-1288,-1612,-6760,6200,13648,421,...,-1549,989,1968,-1524,16360,2480,-677,3366,3260,0


In [3]:
# Spliting the points and the labels
X = dataset.iloc[:, :-1].values  
y = dataset.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [5]:
window = 30 # depends on time window
epochs = 15
batch_size = 256
acc_vec_dim = 6 # depends on how many sensors used

class_names = list(set(y))
num_class = len(class_names)

X_testg = X_test.reshape(X_test.shape[0], window, acc_vec_dim)
X_traing = X_train.reshape(X_train.shape[0], window, acc_vec_dim)
print(X_traing.shape)
print(X_testg.shape)
print(X_traing[0])

(625060, 30, 6)
(156266, 30, 6)
[[ -6884  11255   8449  -3805   1157   -203]
 [ -7854   8611  10340  -6159   -342   -748]
 [ -8707  11575  11793  -2660   2003  -1902]
 [ -5764   6920  12700  -2775   3429  -2854]
 [ -6005   6389  15372   3006   1255  -3440]
 [ -3306   6752  15499   2562    539  -1897]
 [ -3363   8443  13615   3292   1473   1931]
 [ -6249   8837  11193   4661  -4737   -229]
 [ -9218  10061   9085     99  -4329   4200]
 [ -8857   8494  10835  -5377  -1926   6476]
 [-10460  10598  13220  -3851  -3268   5701]
 [ -9042   9105  12321  -1343  -1182   3301]
 [ -6590  10870  12248  -2657  11085   7632]
 [ -7516   8130  10365   1441   8280   3934]
 [ -6227  10810   8100  -3903  10955   5164]
 [ -5991   9998  11717  -5563   9192   3448]
 [ -3013  10184  10585    585   4078    182]
 [ -2774  10222   9542   3192   2880  -3629]
 [ -7940   9365  11836   2269     88   -996]
 [ -4454  11921  10069   2280  -2039  -2983]
 [ -7601   8610  11251   6194 -13753  -5437]
 [ -8451  12010   8679 

In [6]:
input_data = np.array(X_testg, dtype=np.float32)

In [8]:
with open('test_data(input_data).npy', 'wb') as f:
    np.save(f, input_data)
    
with open('test_data(y_test).npy', 'wb') as g:
    np.save(g, y_test)

In [3]:
#SINIIIIIIIIIIIIIIIIIIIIIIIIII
with open('test_data(input_data)9.npy', 'rb') as f:
    input_data = np.load(f)
    
with open('test_data(y_test)9.npy', 'rb') as g:
    y_test = np.load(g)

In [4]:
import time

In [5]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 1000

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

normal = []
tflite = []
same = 0
for i in range(TEST_CASES):
    try:
        t = time.time()
        expected = LSTM_model.predict(input_data[i:i+1])
        normal.append(time.time() - t)
        t = time.time()
        interpreter.set_tensor(input_details[0]["index"], input_data[i:i+1, :, :])
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]["index"])
        tflite.append(time.time() - t)
        
        interpreter.reset_all_variables()

        # Assert if the result of TFLite model is consistent with the TF model.
        np.testing.assert_almost_equal(expected, result)
        #print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")
        same += 1

        # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
        # the states.
        # Clean up internal states.
        #interpreter.reset_all_variables()
    except AssertionError as e:
        #print(e)
        pass

for i in range(TEST_CASES, TEST_CASES*2):
    try:
        
        t = time.time()
        interpreter.set_tensor(input_details[0]["index"], input_data[i:i+1, :, :])
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]["index"])
        tflite.append(time.time() - t)
        
        t = time.time()
        expected = LSTM_model.predict(input_data[i:i+1])
        normal.append(time.time() - t)
        
        interpreter.reset_all_variables()

        # Assert if the result of TFLite model is consistent with the TF model.
        np.testing.assert_almost_equal(expected, result)
        #print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")
        same += 1

        # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
        # the states.
        # Clean up internal states.
        #interpreter.reset_all_variables()
    except AssertionError as e:
        #print(e)
        pass

print(f"Same: {same}")
total = 0
for value in normal:
    total += value
    
total = total/len(normal)
print(f"Normal: {total}")

total = 0
for value in tflite:
    total += value
    
total = total/len(tflite)
print(f"TFLITE: {total}\n")

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_predict_function_43615]

Function call stack:
predict_function -> predict_function -> predict_function


In [28]:
input_data[14:15].shape

(1, 30, 6)

In [29]:
input_details[0]["index"]

0

In [17]:
input_data[i:i+1, :, :]

array([[[ -2384.,  16660.,   1868.,   2875.,   5435.,   7369.],
        [ -2160.,  21632.,   1320.,   3000.,   6167.,   6307.],
        [ -2844.,  16408.,   2404.,   3542.,   7324.,   6017.],
        [ -2356.,  11828.,   1412.,   1814.,   1690.,   2944.],
        [ -2116.,  17960.,     48.,  -1075.,   5847.,   2145.],
        [ -2392.,  15232.,    256.,  -1711.,  15084.,    560.],
        [ -2760.,  14816.,    728.,   -769.,  11125.,  -2087.],
        [ -4656.,  15252.,    600.,   -653.,   7321.,  -3107.],
        [ -5700.,  14840.,    848.,  -1095.,   8354.,  -2022.],
        [ -6144.,  13160.,   1700.,  -1672.,   3973.,  -1570.],
        [ -6800.,  13356.,   2004.,  -1512.,   3831.,  -2308.],
        [-10580.,  12680.,   1536.,  -1066.,   5263.,   -784.],
        [ -9428.,  15680.,   1656.,  -1158.,    261.,    440.],
        [ -5520.,  14880.,   1812.,   -673.,    -82.,  -4973.],
        [ -9736.,  14776.,   2324.,   1766.,  -2742.,  -7363.],
        [ -8828.,  13952.,   1204.,   31

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [13]:
tflite_model2 = converter.convert()

In [14]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model2)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(TEST_CASES):
    try:
        expected = LSTM_model.predict(input_data[i:i+1])
        interpreter.set_tensor(input_details[0]["index"], input_data[i:i+1, :, :])
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]["index"])

        # Assert if the result of TFLite model is consistent with the TF model.
        np.testing.assert_almost_equal(expected, result)
        print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

        # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
        # the states.
        # Clean up internal states.
        interpreter.reset_all_variables()
    except AssertionError as e:
        print(e)

ValueError: Did not get operators, tensors, or buffers in subgraph 1.


In [8]:
# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [10]:
output_details[0]["index"]

56

In [21]:
prediction = []
for i in range(0,1000,3):
    interpreter.set_tensor(input_details[0]["index"], input_data[i:i+3, :, :])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    prediction.append(np.argmax(result[0]))
    prediction.append(np.argmax(result[1]))
    prediction.append(np.argmax(result[2]))
    interpreter.reset_all_variables()
    
accurate_count = 0
for index in range(len(prediction)):
    if prediction[index] == y_test[index]:
        accurate_count += 1
accuracy = accurate_count * 1.0 / len(prediction)

In [22]:
print(accuracy)

0.93812375249501


In [6]:
open("tflite_LSTM.tflite", "wb").write(tflite_model)

178100

In [13]:
input_data[i:i+3, :, :].shape

(3, 30, 6)

In [18]:
interpreter.set_tensor(input_details[0]["index"], input_data[0:3, :, :])
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]["index"])
print(result)
print(np.argmax(result[0]))
print(np.argmax(result[1]))
print(np.argmax(result[2]))
prediction.append(np.argmax(result))
interpreter.reset_all_variables()

[[9.9990237e-01 1.4385454e-10 9.7083503e-05 6.1384912e-07]
 [1.6832739e-05 2.0929762e-04 9.9419832e-01 5.5755591e-03]
 [9.9999785e-01 1.4478806e-13 9.4301823e-11 2.1306262e-06]]
0
2
0
